In [0]:
import numpy as np
from skimage import io, transform, util
from sklearn.metrics import confusion_matrix
import h5py
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, Adam, SGD, Nadam
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Input, merge, ZeroPadding2D
from keras.layers import AveragePooling2D, Concatenate, concatenate
from keras.utils.io_utils import HDF5Matrix
import keras.backend as K

from keras import regularizers

def openHDF5(filename):
    return h5py.File(filename, "a")

def readHDF5(fileName):
    return h5py.File(fileName, 'r')
  
def getHDF5(file, name, qnt, init=0):
    dset = file[name]
    return dset[init:qnt]

def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
    
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())

    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def makeYMat(qnt):
    pets = ['cat', 'dog']
    mat = []
    for i in range(0, qnt):
        for pet in pets:
            if (pet == 'dog'):
                mat.append(1)
            else:
                mat.append(0)   
    return np.array(mat)

In [0]:
path = ""

from google.colab import drive, files
drive.mount('/content/drive/')
path = "/content/drive/My Drive/datacnn/"

dsetTrain = readHDF5(path + 'imgsTrain100R.hdf5')
dsetTest = readHDF5(path + 'imgsTest100R.hdf5')
dsetValid = readHDF5(path + 'imgsValid100R.hdf5')

qnt = 10000

x_train = getHDF5(dsetTrain, 'pets', qnt)
#x_test = getHDF5(dsetTest, 'pets', qnt, init=5000)
#x_test2 = getHDF5(dsetTest, 'pets', 5000)
x_testT = getHDF5(dsetTest, 'pets', 10000)
#x_valid = getHDF5(dsetValid, 'pets', 1496)

#x_train = np.append(x_train, x_test2, axis=0)
y_train = makeYMat(15000//2)
y_trainT = makeYMat(5000)
y_test = makeYMat(5000//2)
y_testT = makeYMat(5000)

BATCH_SIZE = 64
IMAGE_SIZE = 100
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
test_data_gen = ImageDataGenerator(rescale=1./255)
valid_data_gen = ImageDataGenerator(rescale=1./255)
   
#MATX_train = HDF5Matrix(path + 'imgsTrain100R.hdf5', 'pets')
#MATX_test = HDF5Matrix(path + 'imgsTest100R.hdf5', 'pets')
#MATX_valid = HDF5Matrix(path + 'imgsValid100R.hdf5', 'pets')

seed = 0
train_data_gen.fit(x_train, augment=True)

train_gen = train_data_gen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    seed=seed,
    shuffle=True)

test_gen = test_data_gen.flow(
    x_test,
    y_test,
    batch_size=BATCH_SIZE,
    seed=seed,
    shuffle=True)

valid_gen = valid_data_gen.flow(
    x_valid,
    batch_size=BATCH_SIZE,
    shuffle=False)

x_train /= 255
x_test /= 255
x_testT /= 255
x_valid /= 255

#from sklearn.utils import shuffle
#x_train, y_train = shuffle(x_train, y_train, random_state=0)

print("Matrix ready!")

In [0]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/gpu:0'):
  model = Sequential()
  
  model.add(Conv2D(64, (3, 3), activation='linear', input_shape=input_shape))
  model.add(Conv2D(64, (3, 3), activation='linear'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(128, (3, 3), activation='linear'))
  model.add(Conv2D(128, (3, 3), activation='linear'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Conv2D(256, (3, 3), activation='linear'))
  model.add(Conv2D(256, (3, 3), activation='linear'))
  model.add(MaxPooling2D(pool_size=(2, 2)))

  model.add(Flatten())
  model.add(Dense(512, activation='linear', kernel_regularizer=regularizers.l2(0.004))) #Regularizer tirou invariancia do loss
  model.add(Dropout(0.5))
  
  model.add(Dense(512, activation='linear', kernel_regularizer=regularizers.l2(0.004))) # Activation linear possibilitou usar LR maior
  model.add(Dropout(0.5))

  model.add(Dense(1))
  model.add(Activation('sigmoid')) #Sigmoid p/ binary

  model.compile(optimizer=RMSprop(lr=0.0001), #LR Tirou a invariancia das metricas
                loss='binary_crossentropy',
                metrics=[f1_score, 'accuracy', recall, precision])
  print("Model Compiled")

  #print(model.summary())
  model.load_weights(path + 'model16_weights.h5')
  #history = model.fit(x_train, y_train, batch_size=64, epochs=100, verbose=1, validation_data=(x_test, y_test))
  #history = model.fit_generator(train_gen, steps_per_epoch=15000//BATCH_SIZE, epochs=80, validation_data=test_gen, validation_steps=5000//BATCH_SIZE)
  #print("Score Train: " + str(model.evaluate(x_train, y_trainT, batch_size=64)))
  print("Score Test: "  + str(model.evaluate(x_testT, y_testT, batch_size=64)))

In [0]:
metrics = ['acc', 'loss', 'f1_score', 'precision', 'recall']

i = 1
for var in metrics:
  plt.subplot(3,2,i)
  plt.plot(history.history[var])
  plt.plot(history.history['val_'+var])
  plt.title('model '+var)
  plt.ylabel(var)
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper right')
  i += 1

plt.tight_layout()


pre_cls = model.predict(x_test)

for i in range(5000):
  if (pre_cls[i] > 0.5):
    pre_cls[i] = 1
  else:
    pre_cls[i] = 0

cm1 = confusion_matrix(y_test,pre_cls)
print('Confusion Matrix : \n', cm1)
print('TP + TN: ', cm1[0][0]+cm1[1][1])
print('FP + FN: ', cm1[0][1]+cm1[1][0])

pre_cls = model.predict(x_valid)

iDog = 0
iCat = 0
for i in range(1496):
  if (pre_cls[i] > 0.5):
    pre_cls[i] = 1
    iDog += 1
  else:
    pre_cls[i] = 0
    iCat += 1

print('Qnt Cachorro: \n', iDog)
print('Qnt Gato: \n', iCat)

In [0]:
#model.save(path+'model16.h5')
model.save_weights(path + 'model17_weights.h5')

In [0]:
from keras.models import load_model

clas = model
clas.load_weights(path + 'model16_weights.h5')

#history = clas.fit_generator(train_gen, steps_per_epoch=15000//BATCH_SIZE, epochs=100, validation_data=test_gen, validation_steps=5000//BATCH_SIZE)

from sklearn.metrics import roc_curve
y_pred_keras = clas.predict(x_test)
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [0]:
from keras.models import Model
from keras.preprocessing import image
  
clas = model
clas.load_weights(path + 'model16_weights.h5')

img = image.img_to_array(x_train[3])
img = img/255
img = np.expand_dims(img, axis=0)

layers = ['conv2d_63', 'conv2d_64', 'max_pooling2d_31']

for layer in layers:
  print(layer)
  conv2d_output = Model(inputs=clas.input, outputs=clas.get_layer(layer).output)

  conv2d_features = conv2d_output.predict(img)

  fig = plt.figure(figsize=(14,7))
  columns = 8
  rows = 4
  for i in range(columns*rows):
      #img = mpimg.imread()
      fig.add_subplot(rows, columns, i+1)
      plt.axis('off')
      plt.title('filter'+str(i))
      plt.imshow(conv2d_features[0, :, :, i], cmap='gray')
  plt.show()

In [0]:
import csv
import os

path2 = path+'validation'

folder = os.fsencode(path2)

filenames = []
mylist = [['id', 'label']]

i = 0
for file in os.listdir(folder):
  filename = os.fsdecode(file)
  mylist.append([filename[:-4], str(int(pre_cls[i]))])
  i += 1

with open(path + 'submission5.csv', 'w') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_NONNUMERIC)
     for item in mylist:
         wr.writerow(item)